In [1]:
#GDD calculation for corn from class CornGDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class CornGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def con_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


Min / Max temp to calculate GDD for Saint Paul 2024. Here we used GEMS exchage data to calculate GDD

In [5]:
import pandas as pd
StPaul_GEMS = pd.read_csv("st_paul_gems_2023_2024.csv", on_bad_lines='skip')

In [6]:
StPaul_GEMS_2024 = CornGDD(StPaul_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [9]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("5-15-2024").date()
end_date = pd.to_datetime("10-21-2024").date()

Calculating AGDD for corn SaintPaul 2024

In [10]:
corn_gdd_StPaulGEMS = StPaul_GEMS_2024.con_gdd(start_date, end_date)
print(corn_gdd_StPaulGEMS)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
500      64 2024-05-15    6.6   56  448  336      121      920      940   
501      77 2024-05-16   10.6   56  449  337      121      920      942   
502      47 2024-05-17   11.1   56  450  339      121      920      943   
503      44 2024-05-18   10.9   56  451  340      121      920      945   
504      64 2024-05-19    5.1   56  452  341      121      920      946   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
655       0 2024-10-17   -3.6   35  293  156      101      832      578   
656      26 2024-10-18    0.9   34  291  153      101      830      573   
657      86 2024-10-19    4.7   34  288  151      101      828      567   
658      30 2024-10-20   10.4   34  286  149      100      826      561   
659      45 2024-10-21    7.8   33  283  147      100      824      555   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
500     71.96  ...   